In [ ]:
from ped_car_v11 import PedestrianEnv
import numpy as np
from My_DDQN import DDQN
import random
import matplotlib.pyplot as plt

In [ ]:
env=PedestrianEnv()
observation = env.reset()
observation_space=len(observation) #we get the number of parameters in the state
action_space=8 #number of discrete velocities pedestrian can take
agent=DDQN(observation_space, action_space)
episodes=1000

In [ ]:
reward_list=[0]
print(agent.gamma)
for e in range(episodes):
    episode_reward=0
    state=env.reset()
    state = np.reshape(state, [1, observation_space])    
    while True:
        action=agent.act(state)        
        #action_1=action*((env.P_velocity_limit)/(action_space-1))
        action_1=-1*env.P_velocity_limit+action*(2*(env.P_velocity_limit)/(action_space-1))
        state_next, reward, terminal = env.step(np.array([action_1]))
        episode_reward+=reward
        state_next = np.reshape(state_next, [1, observation_space])
        agent.add_memory(state, action, reward, state_next, terminal)
        if(reward==10000):            
            agent.add_bad_memory(state, action, reward, state_next, terminal)
        state = state_next
        if(len(agent.memory)>agent.minibatch_size):
            agent.replay()
        if terminal:
            print ("Episode: " + str(e) + ", exploration: " + str(agent.exploration_rate) + ", score: " + str(episode_reward)+" ,goal reward: "+str(reward))            
            agent.target_model_update()
            reward_list.append((reward_list[len(reward_list)-1]*e+episode_reward)/(e+1))
            break
    if agent.exploration_rate>agent.exploration_min:
        agent.exploration_rate*=agent.exploration_decay       
    

In [ ]:
plt.figure(1)
plt.plot(reward_list)
plt.show()

In [ ]:
state=env.reset()
state = np.reshape(state, [1, observation_space])
death_toll=0
safe_chicken=0
done_count=0
agent.exploration_rate=0
for t in range(5000): 
    action = agent.act(state)
    print(action)
    action_1=-1*env.P_velocity_limit+action*(2*(env.P_velocity_limit)/(action_space-1))    
    observation, reward, done = env.step(np.array([action_1]))          
    if done:       
        done_count+=1
        if (reward==-10000):
            death_toll+=1                                              
        if (reward==10000):           
            safe_chicken+=1        
        env.reset()
    state = np.reshape(observation, [1, observation_space])
    
    
print('Death_toll '+str(death_toll))
print('Episodes', done_count)
print('Death_toll % '+str(death_toll*100/(done_count)))
print('Safe_chicken',safe_chicken)
print('Safe_toll % '+str(safe_chicken*100/(done_count)))

In [ ]:
N = 20
cumsum, moving_aves = [0], []

for i, x in enumerate(reward_list, 1):
    cumsum.append(cumsum[i-1] + x)
    if i>=N:
        moving_ave = (cumsum[i] - cumsum[i-N])/N
        #can do stuff with moving_ave here
        moving_aves.append(moving_ave)

plt.figure(2)
plt.plot(moving_aves)
plt.show()

In [ ]:
from keras.models import load_model
agent.model.save('ddqn_ped_v11.h5')

In [ ]:

from matplotlib.animation import FuncAnimation

death_toll=0
safe_chicken=0
done_count=0
count=0
Ped_Pos=[]
Car_xPos=[]
Car_yPos=[]
d = env.d
W = env.W

env = PedestrianEnv()
episodes = 3
for e in range(episodes):
	state=env.reset()
	state = np.reshape(state, [1, observation_space])
	while True:	
		action=agent.act(state)        
		action_1=-1*env.P_velocity_limit+action*(2*(env.P_velocity_limit)/(action_space-1))
		state_next, reward, done = env.step(np.array([action_1]))		
		Ped_Pos.append(state_next[0])
		Car_xPos.append(state_next[1])
		Car_yPos.append(state_next[2])
		state_next = np.reshape(state_next, [1, observation_space])
		state=state_next	    
		if done:
			done_count+=1
			if (reward==-10000):
				death_toll+=1				
			if (reward==5000):
				safe_chicken+=1
			break			
			

#Plot initialization
fig, ax = plt.subplots()
ax.set_xlim(0, 1.8*d)
ax.set_ylim(0, W+2)
xdata, ydata = [], []
ln1, = plt.plot([], [], 'ro',markersize=20)
ln2, = plt.plot([], [], 'bx',markersize=20)

#Animate Car
def update1(frame1):
    ln1.set_data(Car_xPos[frame1], Car_yPos[frame1])
    return ln1,

#Animate Ped
def update2(frame2):
    ln2.set_data(d, Ped_Pos[frame2])
    return ln2,

def updateALL(frame):
    a = update1(frame)
    b = update2(frame)
    print(frame)
    return a+b

#Animate
iterations=len(Ped_Pos)
plt.grid()
ani = FuncAnimation(fig, updateALL, frames= iterations,
                    blit=True,repeat=False)
plt.pause(1)
plt.show()

#Results
# print('Episodes', done_count)
# print('Safe_chicken',safe_chicken)
# print('Death_toll '+str(death_toll))
# print('Death_toll % '+str(death_toll*100/(death_toll+safe_chicken)))
ani.save('animation_back_1.gif', writer='imagemagick', fps=60)